# Import libraries

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )
# Basic libraries
import mlflow
import os
import numpy    as np
import pandas   as pd

# Parameters
test_size = 0.1
seed = 42


# Loading data

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('Data/dataset.csv').dropna()

df_train, df_test = train_test_split(df, test_size=test_size, stratify=df['Class'], random_state=seed)

# Get X & y
trainX = df_train.iloc[:, :-1].values
trainY = df_train.iloc[:,  -1].values.astype('int')

testX = df_test.iloc[:, :-1].values
testY = df_test.iloc[:,  -1].values.astype('int')

# Load model

In [ ]:
# Initiate mlflow server
# Command: mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 127.0.0.1 --port 5000
# 
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Set path of requested model
logged_model = 'runs:/fc2964bd2aa344218b5a12a49b256d19/models'

# Load model as a PyFuncModel.
model = mlflow.sklearn.load_model(logged_model)

# Optimized model & Explainability modules

In [ ]:
# Classification report
from sklearn.metrics import classification_report

pred = model.predict(testX)
print(classification_report(testY, pred))

## Explainability

### Feature importance

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4,6))
plt.barh(df.columns[:-1], model.feature_importances_)

## LIME

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
import lime
import lime.lime_tabular
explainer = lime.lime_tabular.LimeTabularExplainer(trainX, 
                                                   feature_names=df.columns, 
                                                   class_names=['Class:0', 'Class:1','Class:2','Class:3'], 
                                                   verbose=True, 
                                                   mode='classification')

In [ ]:
exp = explainer.explain_instance(trainX[12], model.predict_proba, num_features=5)
exp.show_in_notebook(show_table=True)

## SHAP

In [ ]:
import shap
explainer = shap.TreeExplainer(model=model, feature_names=df.columns[:-1])
shap_values = explainer.shap_values(trainX)

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, df.iloc[:,:-1], plot_type="bar")

In [ ]:
shap.dependence_plot('Feature-1', 
                      shap_values=shap_values[2], 
                      features=trainX,
                      feature_names=df.columns[:-1])